In [13]:
# import packages needed
import itertools
import numpy as np

from typing import Iterable, Callable, Mapping, TypeVar, List
from rl.markov_process import TransitionStep
from rl.returns import returns
from rl.function_approx import Tabular
from rl.iterate import last
from collections import defaultdict

from rl.chapter2.simple_inventory_mrp import SimpleInventoryMRPFinite, InventoryState
from rl.monte_carlo import mc_prediction
from rl.td import td_prediction

from rl.distribution import Constant

In [2]:
S = TypeVar('S')

### Problem 1 & 3: Tabular MC Prediction from scratch

In [3]:
def tab_mc_prediction(trs: Iterable[Iterable[ReturnStep[S]]],
                      weight_func: Callable[[int],float],
                      gamma: float,
                      tol: float = 1e-06) -> List[Mapping[S,float]]:
    
    episodes = [returns(trace,gamma,tol) for trace in trs]
    
    v: List[Mapping[S,float]] = []
    occurence: Mapping[S,int] = defaultdict(lambda:0)
        
    for episode in episodes:
        if len(v) == 0:
            curr_v : Mapping[S,float] = defaultdict(lambda:0)
        else:
            curr_v : Mapping[S,float] = {k:var for (k,var) in v[-1].items()}
        for st in episode:
            occurence[st.state] += 1
            curr_v[st.state] = curr_v[st.state]*(1-weight_func(occurence[st.state])) +\
                weight_func(occurence[st.state])*st.return_
        v.append(curr_v)
        
    return v

#### Verify Correctness

In [4]:
user_capacity = 2
user_poisson_lambda = 1.0
user_holding_cost = 1.0
user_stockout_cost = 10.0

user_gamma = 0.9

si_mrp = SimpleInventoryMRPFinite(
    capacity=user_capacity,
    poisson_lambda=user_poisson_lambda,
    holding_cost=user_holding_cost,
    stockout_cost=user_stockout_cost
)

start = InventoryState(on_hand = 0, on_order = 0)
sample = [list(itertools.islice(si_mrp.simulate_reward(Constant(start)),1000)) 
          for _ in range(100)]

In [5]:
res_1 = tab_mc_prediction(sample, lambda n: 1./n, user_gamma)[-1]

In [6]:
approx = Tabular()
res_2 = last(mc_prediction(sample,approx,user_gamma)).values_map

In [7]:
for k in res_1:
    print(f"State {k}: %.4f vs. %.4f"%(res_1[k],res_2[k]))

State InventoryState(on_hand=0, on_order=0): -35.3453 vs. -35.3453
State InventoryState(on_hand=0, on_order=2): -28.1484 vs. -28.1484
State InventoryState(on_hand=2, on_order=0): -30.2016 vs. -30.2016
State InventoryState(on_hand=1, on_order=0): -28.8796 vs. -28.8796
State InventoryState(on_hand=0, on_order=1): -28.0361 vs. -28.0361
State InventoryState(on_hand=1, on_order=1): -29.3778 vs. -29.3778


### Problem 2 & 3: Tabular TD Prediction from scratch

In [8]:
def tab_td_prediction(trans: Iterable[TransitionStep[S]],
                      weight_func: Callable[[int],float],
                      gamma: float) -> List[Mapping[S,float]]:
    
    v: Mapping[S,float] = defaultdict(lambda:0)
    occurence: Mapping[S,int] = defaultdict(lambda:0)
    all_v: List[Mapping[S,float]] = []
        
    for st in trans:
            occurence[st.state] += 1
            v[st.state] += weight_func(occurence[st.state])*(st.reward + gamma*v[st.next_state]
                                                             - v[st.state])
            all_v.append(v)

    return all_v

#### Verify correctness

In [9]:
start = InventoryState(on_hand = 0, on_order = 0)
sample = list(itertools.islice(si_mrp.simulate_reward(Constant(start)),100000))

In [10]:
res_1 = tab_td_prediction(sample, lambda n: 1./n, user_gamma)[-1]

In [11]:
approx = Tabular()
res_2 = last(td_prediction(sample,approx,user_gamma)).values_map

In [12]:
for k in res_1:
    print(f"State {k}: %.4f vs. %.4f"%(res_1[k],res_2[k]))

State InventoryState(on_hand=0, on_order=0): -25.4490 vs. -25.4490
State InventoryState(on_hand=0, on_order=2): -18.2953 vs. -18.2953
State InventoryState(on_hand=1, on_order=0): -18.9741 vs. -18.9741
State InventoryState(on_hand=0, on_order=1): -17.9702 vs. -17.9702
State InventoryState(on_hand=1, on_order=1): -19.3000 vs. -19.3000
State InventoryState(on_hand=2, on_order=0): -20.3084 vs. -20.3084


### Problem 4: Extend RandomWalkMRP